In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction


Hello everyone. In this notebook I am going to use self-attention mechanism based transformers to do the AI generated text detection.

# Loading of Datasets

I am going to use the giant 400k dataset which is available on kaggle.For low computation purposes we are going to only use 90000 rows of it, and concatenate it with the orignal dataset.As the orignal dataset is very unbalanced (1375(0) and 3(1)) so we use equal 45000,45000 rows from the huge dataset for the input.If we pass the imbalanced dataset to the 
model ,it will get biased towards the majority class, which affect our model performance, also transformers works more well if we feed them with more data

In [ ]:
train1=pd.read_csv('/kaggle/input/augmented-data-for-llm-detect-ai-generated-text/final_train.csv')

In [ ]:
train1.shape 

In [ ]:
train2=pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')

In [ ]:
test1=pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [ ]:
test2=pd.read_csv('/kaggle/input/augmented-data-for-llm-detect-ai-generated-text/final_test.csv')

In [ ]:
test2.shape

In the giant dataset the target column is *label* , so I am going to create a new column *generated* and its values are copied from *label* column and then I drop the *label* column

In [ ]:
train1['generated']=train1['label']

In [ ]:
train1.drop(['label'],axis=1,inplace=True)

In [ ]:
import plotly.express as px

# Assuming 'generated' is the column name in your 'train1' DataFrame
# Replace 'train1' with the actual DataFrame variable
fig = px.histogram(train1, x='generated', title='Distribution of Generated Column in the large dataset',
                   labels={'generated': 'Generated Column', 'count': 'Count'},
                   color='generated',  # Use the 'generated' column for color
                   color_discrete_map={'0': 'yellow', '1': 'red'},  # Set colors for 0 and 1
                   category_orders={'generated': ['0', '1']})
                    # Display additional information on hover

# Show the plot
fig.show()



In [ ]:
fig = px.histogram(train2, x='generated', title='Distribution of Generated Column in the orignal dataset',
                   labels={'generated': 'Generated Column', 'count': 'Count'},
                   color='generated',  # Use the 'generated' column for color
                   color_discrete_map={'0': 'yellow', '1': 'red'},  # Set colors for 0 and 1
                   category_orders={'generated': ['0', '1']})
                    # Display additional information on hover

# Show the plot
fig.show()

I am droping the prompt_id ,id column for the model training.

In [ ]:
train2.drop(['id','prompt_id'],axis=1,inplace=True)


In [ ]:
original_df = train1

# Separate AI-generated and human-generated texts
ai_generated_df = original_df[original_df['generated'] == 1].sample(n=45000, random_state=42)
human_generated_df = original_df[original_df['generated'] == 0].sample(n=45000, random_state=42)

# Concatenate the two dataframes
sampled_df = pd.concat([ai_generated_df, human_generated_df])

# Shuffle the dataset
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
sampled_df

In [ ]:
df1=pd.concat([train2,sampled_df])

In [ ]:
fig = px.pie(df1, names='generated', title='Distribution of Generated Column',
             labels={'generated': 'Generated Column'},
             color_discrete_map={'0': 'yellow', '1': 'red'})
fig.show()

Now it is balanced.

In [ ]:
df1

In [ ]:
df1.drop_duplicates(inplace=True,ignore_index=True)

In [ ]:
df1.reset_index(inplace=True)

In [ ]:
df1.drop(['index'],axis=1,inplace=True)

In [ ]:
df1

# Text Preprocessing && Tokenization

In [ ]:
import re # regular expression library
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
def normalize(text):
    # Replace with whitespace to separate '😃\n\nFor'
    text = text.replace(r"\n", r" ")
    text = text.replace(r"\r", r" ")
    # Drop puntuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra spaces from '😃  For' to '😃 For'
    text = re.sub(r"\s+", r" ", text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove leading and trailing whitespace
    text = text.strip()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    #lower case the all capital alphabets
    text=text.lower()
    stop_words = set(stopwords.words('english'))
    word_tokens=word_tokenize(text)
    text = ' '.join([word for word in word_tokens if word not in stop_words])
    return text

df1['text'] = df1['text'].apply(lambda x: normalize(x))
test1['text'] = test1['text'].apply(lambda x: normalize(x))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df1['text'])# gives a index on each word
b1=len(tokenizer.word_index)# total no of words

In [ ]:
b1

In [ ]:
a=tokenizer.texts_to_sequences(df1['text'])
maxi=max(len(x) for x in a)# longest sentence size

In [ ]:
maxi

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
a1=pad_sequences(a,maxlen=maxi,padding='post')

In [ ]:
df1['text']=a1.tolist()

In [ ]:
import numpy as np
X = np.vstack(df1['text'].to_numpy())
y=np.vstack(df1['generated'].to_numpy())

In [ ]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,shuffle=True)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Model(Transformers)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Reshape,Embedding, Layer, Dense, Dropout, MultiHeadAttention, LayerNormalization, Input, GlobalAveragePooling1D,UpSampling1D
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, heads, neurons):
        super(TransformerEncoder, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [layers.Dense(neurons, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(0.5)
        self.dropout2 = layers.Dropout(0.5)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
embed_dim = 10
heads = 10
neurons = 32
maxlen = maxi
vocab_size = b1+1

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerEncoder(embed_dim, heads, neurons)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = Dropout(0.20)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,y_train,
                    validation_split=0.2,
                    epochs=5,
                    batch_size=32,
                    )

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

**Testing on the test part of the huge dataset of the model**

In [ ]:
# test_sequences = tokenizer.texts_to_sequences(test2['text'])
# X_test_padded = pad_sequences(test_sequences, maxlen=maxi)  # Adjust maxlen based on your model

# Make predictions using the trained model
# predictions = model.evaluate(X_test_padded,test2['label'],verbose=0)

In [ ]:
# print("Accuracy: %.2f%%" % (predictions[1]*100))

# Submission File Creation

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test1['text'])
X_test_padded = pad_sequences(test_sequences, maxlen=maxi)  # Adjust maxlen based on your model

# Make predictions using the trained model
predictions = model.predict(X_test_padded)

# Assuming 'id' is the column containing unique identifiers in the test set
# Create a DataFrame to store the results
results_df = pd.DataFrame({'id': test1['id'], 'generated': predictions.flatten()})

# Save the results to a CSV file
results_df.to_csv('submission.csv', index=False)


In [ ]:
final=pd.read_csv('submission.csv')
final.head()

# Keypoints


The key and important insights from this competition which I get are below:
* The dataset is imbalanced very much ,so we have to balance it by upsampling or using external dataset.In my case I have used a external dataset of kaggle
* I have also seen that there are many typos and grammatical errors in the train dataset. So when you used external dataset which are correct and majority in class(90K vs 1378) then thw high and advanced LLMs overfits the training data ,you get a high level of accuracy . But when you used this model for the hidden test dataset you get 0.85 around LB score due to the typos available in the hidden test data ,infact due to this typos the LLMs are become unable to that wrong semantics of that data.
* Obviously in this case Tf-Idf with simple classification model overkills this LLMs because they doesn't focus on the semantics of the text ,they just do the count / frequency of the word, so they get good LB as well as accuracy.
* But in real life scenario definitely this high LB scored models don't get good results without focusing on the semantics ,definitely the LLMs traditionally overkills this simple models.
 